In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean_actors,
    decode_clean,

)
import explo_data_analysis.eda_movies as eda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# link = "big_dataframe/actors.parquet"
link = "movies_datasets/name_basics.tsv"
test = import_datasets(link, 'pandas', sep="\t")
test

In [ ]:
df = test.copy()

In [ ]:
df

In [ ]:
df.drop(["deathYear", "primaryProfession"], axis=1, inplace=True)

In [ ]:
clean = DataCleaner()
clean.fix_values(df, "fix_n")

In [ ]:
df

In [ ]:
df["knownForTitles"] = np.where(df["knownForTitles"] == 0, "Unknown", df["knownForTitles"])
df["knownForTitles"] = df["knownForTitles"].str.split(",")
# mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")


In [ ]:
df["birthYear"] = df["birthYear"].astype("int64")

In [ ]:
df.head()

In [ ]:
df.to_parquet("clean_datasets/person.parquet")

In [ ]:
# link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

# movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

In [ ]:
df_prin = principals.copy()

In [ ]:
df_prin.drop(["job"], inplace=True, axis=1)


In [ ]:
df_prin

In [ ]:
df_prin = clean.fix_values(df_prin, "fix_n")


In [ ]:

df_prin["characters"] = np.where(df_prin["characters"] == 0, "Unknown", df_prin["characters"])
# df_prin["characters"] = df_prin["characters"].str.split(",")
df_prin["characters"] = df_prin["characters"].apply(decode_clean_actors).str.split(",")


In [ ]:
df_prin["category"].unique()

In [ ]:

test = df_prin.copy()

In [ ]:
actors_list = ["self", "actor", "actress"]

actors = test[test['category'].isin(actors_list)]

# actors = df_prin["category"].str.contains("act")
# directors = df_prin["category"].str.contains("dire")
# writors = df_prin["category"].str.contains("wri")
# cinemato = df_prin["category"].str.contains("cinem")

In [ ]:
directors = test[test["category"].str.contains("director")]

In [ ]:
actors

In [ ]:
directors

In [ ]:
actors.to_parquet("clean_datasets/actors.parquet")

In [ ]:
directors.to_parquet("clean_datasets/directors.parquet")

In [ ]:
# j'ai essayé de lancer ça ça a pris plus de 7 min j'ai give up

others = test[~test["category"].isin([actors, directors])]

In [ ]:
df.dtypes

In [ ]:
actors.dtypes

In [ ]:
directors.dtypes

In [ ]:
link = "clean_datasets/movies_cleaned.parquet"

movies = import_datasets(link, "parquet")

In [ ]:
df_movies = movies.copy()

In [ ]:
df_actors = actors.copy()

In [ ]:
df_person = df.copy()

In [ ]:
df_actors.head()

In [ ]:
df_person.head()

In [ ]:
actors_names = pd.merge(
    df_actors,
    df_person,
    on = "nconst"
)

In [ ]:
actors_names.head()

In [ ]:
movies_actors = pd.merge(
    actors_names,
    df_movies,
    left_on = "tconst",
    right_on = "titre_id"
)

In [ ]:
movies_actors.head()

In [ ]:
movies_actors.drop(["tconst"], inplace=True, axis=1)

In [ ]:
copy = movies_actors.copy()

In [ ]:
column_order = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "rating_avg",
    "rating_votes",
    "original_language",
    "original_title",
    "popularity",
    "production_countries",
    "revenue",
    "spoken_languages",
    "status",
    "titleId",
    "region",
    "cuts",
    "nconst",
    "category",
    "characters",
    "primaryName",
    "birthYear",
    "knownForTitles",
    "ordering"
]

df = copy[column_order]

df.head()

In [ ]:
df.head()

In [ ]:
df[df["primaryName"].str.contains("rard Depardieu")]

In [ ]:
df["primaryName"].value_counts()

**JME SUIS ARRETÉ LA**


In [ ]:
# link = "big_dataframe/actors.parquet"
# df_og = import_datasets(
#     link,
#     types="parquet",
# )
# cleaning = DataCleaner()
# df = cleaning.fix_values(df_og, "fix_n")
# # df = cleaning.fix_values(df1, "fix_encode")

In [ ]:
df["person_birthdate"] = df["person_birthdate"].astype("int64")

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df["person_film"] = df["person_film"].str.split(",")

In [ ]:
df["person_role"] = np.where(df["person_role"] == 0, "Unknown", df["person_role"])

In [ ]:
df.isna().sum()

In [ ]:
df["person_film"] = np.where(df["person_film"].isna(), "Unknown", df["person_film"])

In [ ]:
mf = df.copy()

In [ ]:
mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")

In [ ]:
mf.head()

In [ ]:
mf[mf["person_name"].str.contains('Eddie Murphy')]

In [ ]:
mf.shape

In [ ]:
# merge avec movies

In [ ]:
link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

In [ ]:
principals = principals[["tconst", "nconst"]]

principals.head()

In [ ]:
movies_with_personid = pd.merge(
    movies_cleaned,
    principals,
    left_on = "titre_id",
    right_on = "tconst",
)
#  ICi on récuperer tous les acteurs avec leurs titres

In [ ]:
condi2 = (
    (movies_with_personid["titre_id"].str.contains("tt0068991")) &
    (movies_with_personid["nconst"].str.contains("nm0000367"))
)
movies_with_personid[condi2]

In [ ]:
movies_with_personid["titre_id"].unique().size

In [ ]:
movies_with_personid

In [ ]:
actors_merged = pd.merge(
    mf,
    movies_with_personid,
    left_on = "person_id",
    right_on = "nconst",
    how="left"
)

gerard = movies_with_personid[movies_with_personid.nconst.isin(mf.person_id)]
# gerard.head()
# actors_merged = actors_merged.drop("nconst", axis=1)

In [ ]:
gerard

In [ ]:
actors_merged = pd.merge(
    gerard,
    mf,
    left_on = "nconst",
    right_on = "person_id",
)

In [ ]:
actors_merged

In [ ]:
actors_merged["titre_id"].unique().size

In [ ]:
condi2 = (
    (actors_merged["titre_id"].str.contains("tt0068991")) &
    (actors_merged["nconst"].str.contains("nm0000367"))
)
actors_merged[condi2]

In [ ]:
df = actors_merged.copy()

In [ ]:
df.head()

In [ ]:
condi2 = (
    (principals["tconst"].str.contains("tt0068991")) &
    (principals["nconst"].str.contains("nm0000367"))
)


In [ ]:
len(df[df["person_name"].str.contains("Gérard Depardieu")])

In [ ]:
df.isna().sum()